In [1]:
import pandas as pd
import matplotlib.pyplot as plt

## Dataset overview

In [2]:
# Load data
df = pd.read_csv("../data/processed/master_file.csv")

# Overview
print(df.shape)
print(df.index.min(), df.index.max())
df.head()

(6397, 84)
0 6396


,date,EPU_CAN,IP_CAN,10YS_CAN,INF_YoY_CAN,EX_CAN,IM_CAN,RECESS_CAN,GDP_CAN,GDPC_CAN,...,Unnamed: 0,CCI_CHI,CCI_GER,CCI_JAP,CCI_MEX,YS_CAN,YS_MEX,YS_JAP,YS_CHI,YS_GER
0,1914-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1914-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1914-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1914-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1914-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.columns

Index(['date', 'EPU_CAN', 'IP_CAN', '10YS_CAN', 'INF_YoY_CAN', 'EX_CAN',
       'IM_CAN', 'RECESS_CAN', 'GDP_CAN', 'GDPC_CAN', 'EXR_CAN', 'CCI_CAN',
       'UNEMP_CAN', 'INF_CAN', 'EPU_GER', 'IP_GER', 'INF_YoY_GER', 'UNEMP_GER',
       'IM_GER', 'EX_GER', 'RECESS_GER', 'GDP_GER', 'GDPC_GER', '10YS_GER',
       'EXR_GER', 'INF_GER', 'EPU_JAP', 'IP_JAP', 'INF_YoY_JAP', 'UNEMP_JAP',
       'EX_JAP', 'RECESS_JAP', 'GDP_JAP', 'GDPC_JAP', '10YS_JAP', 'IM_JAP',
       'EXR_JAP', 'INF_JAP', 'EPU_USA', 'IP_USA', 'YS_USA', 'INF_YoY_USA',
       'UNEMP_USA', 'EX_USA', 'IM_USA', 'RECESS_USA', 'GDP_USA', 'GDPC_USA',
       'CCI_USA', 'INF_USA', 'EPU_MEX', 'IP_MEX', 'INF_YoY_MEX', 'GDP_MEX',
       '2YS_MEX', '1OYS_MEX', 'EXR_MEX', 'UNEMP_MEX', 'GDPC_MEX', 'IM_MEX',
       'EX_MEX', 'RECESS_MEX', 'INF_MEX', 'EPU_CHI', 'IP_CHI', 'INF_CHI',
       'IM_CHI', 'EX_CHI', 'RECESS_CHI', '10YS_CHI', 'EXR_CHI', 'UNEMP_CHI',
       'GDPC_CHI', 'GDP_CHI', 'Unnamed: 0', 'CCI_CHI', 'CCI_GER', 'CCI_JAP',
       'C

### Drop residual columns


In [4]:
df = df.drop(columns=["Unnamed: 0", "10YS_CHI", "10YS_JAP", '2YS_MEX', '1OYS_MEX', "10YS_CAN", "10YS_GER"])

###  Clean and Normalize Dates


In [5]:
df["date"] = pd.to_datetime(df["date"]).dt.to_period("M").dt.to_timestamp()


In [6]:
# Sort and set date as index
df = df.sort_values("date").set_index("date")

In [7]:
# Remove any duplicate rows
df = df.drop_duplicates()
print(f"Remaining rows: {len(df)}")

Remaining rows: 6375


In [8]:
df.to_csv("../data/processed/clean_master_file.csv")

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6375 entries, 1914-01-01 to 2025-06-01
Data columns (total 76 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   EPU_CAN      5532 non-null   float64
 1   IP_CAN       5505 non-null   float64
 2   INF_YoY_CAN  474 non-null    float64
 3   EX_CAN       1904 non-null   float64
 4   IM_CAN       1904 non-null   float64
 5   RECESS_CAN   5156 non-null   float64
 6   GDP_CAN      1925 non-null   float64
 7   GDPC_CAN     194 non-null    float64
 8   EXR_CAN      5710 non-null   float64
 9   CCI_CAN      3707 non-null   float64
 10  UNEMP_CAN    5892 non-null   float64
 11  INF_CAN      5451 non-null   float64
 12  EPU_GER      5436 non-null   float64
 13  IP_GER       454 non-null    float64
 14  INF_YoY_GER  474 non-null    float64
 15  UNEMP_GER    834 non-null    float64
 16  IM_GER       1805 non-null   float64
 17  EX_GER       1717 non-null   float64
 18  RECESS_GER   5156 non-null   f

### Utility Functions

###  `get_aligned_subset(df, cols)`

This function ensures that all selected columns in a DataFrame are **aligned to start at the same time**, based on the **latest first valid (non-NaN) entry** across those columns.

####  How it works:
- For each column in `cols`, it finds the **first date with valid data**.
- Then, it picks the **latest** of these starting dates.
- The DataFrame is then **trimmed from that date forward**, ensuring all selected columns have complete data from that point on.

####  Example Use Case:
Use this when creating time-aligned plots or analyses (e.g. scatter plots or rolling correlations) to avoid distortions from missing early data in some variables.

#### Raises:
- `ValueError` if none of the columns have any valid data.

---

In [10]:
def get_aligned_subset(df, cols):
    # Collect all valid start dates
    start_dates = [
        df[col].first_valid_index()
        for col in cols
        if df[col].first_valid_index() is not None
    ]

    # Error handling: no valid data
    if not start_dates:
        raise ValueError("None of the specified columns have valid data.")

    # Find the latest start date
    latest_start = max(start_dates)

    # Trim the subset
    subset = df[cols].copy()
    subset = subset[subset.index >= latest_start]

    return subset

### `get_aligned_df(df, prefixes)`

This function selects and aligns a subset of columns in a DataFrame based on a list of prefixes.

#### How it works:
- It searches for all column names in the DataFrame that start with any of the specified prefixes (e.g., `"GDP_"`, `"UNEMP"`, `"RECESS_"`).
- It creates a subset DataFrame with only those columns.
- Then it calls `get_aligned_subset(...)` to align the data, trimming all columns to start at the same time (i.e., the latest first valid observation across all selected columns).

#### Use Case:
Use this function when you want to analyze relationships between multiple macroeconomic variables (e.g., GDP and EPU) but need to ensure all series start at the same point in time.


In [11]:
def get_aligned_df(df, prefixes):
    """Extracts and aligns columns by list of prefixes (e.g. ['GDP_', 'RECESS_'])"""
    cols = [col for col in df.columns if any(col.startswith(p) for p in prefixes)]
    subset = df[cols].copy()
    aligned = get_aligned_subset(subset, cols)
    return aligned


### `describe_by_prefix(df, prefix, round_digits=2)`

This function prints and returns summary statistics (`.describe().T`) for all DataFrame columns that start with a given prefix.

#### How it works:
- Scans column names in `df` and selects those that start with the provided `prefix`.
- If any matching columns are found, it:
  - Computes summary statistics (`count`, `mean`, `std`, `min`, `25%`, `50%`, `75%`, `max`)
  - Prints the prefix name
  - Displays the summary rounded to the specified number of decimal places
  - Returns the summary DataFrame for optional further use (e.g., saving to Excel)

####  Note:
If no matching columns are found, it prints a warning and returns `None`.

In [12]:
def describe_by_prefix(df, prefix, round_digits=2):
    """Prints prefix name and shows summary statistics for matching columns"""
    cols = [col for col in df.columns if col.startswith(prefix)]
    if not cols:
        print(f"No columns found for prefix: {prefix}")
        return
    print(f"\nSummary for prefix: {prefix}")
    summary = df[cols].describe().T
    display(summary.round(round_digits))
    return summary

###  Aligning Variable Start Dates

Next, we'll check when each variable starts reporting data, so we can trim the dataset to a shared starting period across all selected indicators.


In [13]:
aligned_gdp_df = get_aligned_df(df, ["GDP_", "RECESS_", "EPU_"])
display(aligned_gdp_df.head())

,EPU_CAN,RECESS_CAN,GDP_CAN,EPU_GER,RECESS_GER,GDP_GER,EPU_JAP,RECESS_JAP,GDP_JAP,EPU_USA,RECESS_USA,GDP_USA,EPU_MEX,GDP_MEX,RECESS_MEX,EPU_CHI,RECESS_CHI,GDP_CHI
date,,,,,,,,,,,,,,,,,,
1995-01-01,194.27143,1.0,314818.8,107.05698,1.0,5388.914,95.93575,0.0,458231.3,136.11,1.0,11319.951,0.328965,3518.95475,1.0,192.91191,1.0,1831.415106
1995-02-01,126.45159,1.0,NaN,133.15922,1.0,NaN,69.55904,0.0,NaN,134.42,1.0,NaN,NaN,NaN,1.0,193.98785,1.0,NaN
1995-03-01,162.71977,1.0,NaN,98.76192,1.0,NaN,102.55873,0.0,NaN,97.52,1.0,NaN,NaN,NaN,1.0,88.22704,1.0,NaN
1995-04-01,128.56147,1.0,314952.5,78.15823,1.0,5432.804,66.37704,0.0,NaN,69.45,1.0,11353.721,0.000000,3306.50125,1.0,131.03471,1.0,NaN
1995-05-01,96.17798,1.0,NaN,57.67824,1.0,NaN,93.31389,0.0,NaN,64.41,1.0,NaN,NaN,NaN,1.0,177.09686,1.0,NaN


In [14]:
aligned_gdpc_df = get_aligned_df(df, ["GDPC_", "RECESS_", "EPU_"])
display(aligned_gdpc_df.head())

,EPU_CAN,RECESS_CAN,GDPC_CAN,EPU_GER,RECESS_GER,GDPC_GER,EPU_JAP,RECESS_JAP,GDPC_JAP,EPU_USA,RECESS_USA,GDPC_USA,EPU_MEX,GDPC_MEX,RECESS_MEX,EPU_CHI,RECESS_CHI,GDPC_CHI
date,,,,,,,,,,,,,,,,,,
1995-01-01,194.27143,1.0,31562.0,107.05698,1.0,31672.0,95.93575,0.0,31340.0,136.11,1.0,42654.0,0.328965,7970.640385,1.0,192.91191,1.0,609.604338
1995-02-01,126.45159,1.0,NaN,133.15922,1.0,NaN,69.55904,0.0,NaN,134.42,1.0,NaN,NaN,NaN,1.0,193.98785,1.0,NaN
1995-03-01,162.71977,1.0,NaN,98.76192,1.0,NaN,102.55873,0.0,NaN,97.52,1.0,NaN,NaN,NaN,1.0,88.22704,1.0,NaN
1995-04-01,128.56147,1.0,NaN,78.15823,1.0,NaN,66.37704,0.0,NaN,69.45,1.0,42660.0,0.000000,NaN,1.0,131.03471,1.0,NaN
1995-05-01,96.17798,1.0,NaN,57.67824,1.0,NaN,93.31389,0.0,NaN,64.41,1.0,NaN,NaN,NaN,1.0,177.09686,1.0,NaN


In [15]:
aligned_unemp_df = get_aligned_df(df, ["UNEMP", "RECESS_", "EPU_"])
display(aligned_unemp_df.head())

,EPU_CAN,RECESS_CAN,UNEMP_CAN,EPU_GER,UNEMP_GER,RECESS_GER,EPU_JAP,UNEMP_JAP,RECESS_JAP,EPU_USA,UNEMP_USA,RECESS_USA,EPU_MEX,UNEMP_MEX,RECESS_MEX,EPU_CHI,RECESS_CHI,UNEMP_CHI
date,,,,,,,,,,,,,,,,,,
2005-01-01,45.85582,0.0,6.9,48.37984,5.509729,1.0,54.28121,4.633333,0.0,66.99,5.3,0.0,0.060049,3.99122,0.0,75.73044,1.0,4.52
2005-01-01,45.85582,0.0,6.9,48.37984,5.509729,1.0,54.28121,4.633333,0.0,66.99,5.3,0.0,0.060049,3.99122,0.0,75.73044,1.0,4.52
2005-01-01,45.85582,0.0,6.9,48.37984,5.509729,1.0,54.28121,4.633333,0.0,66.99,5.3,0.0,0.060049,3.99122,0.0,75.73044,1.0,4.52
2005-01-01,45.85582,0.0,6.9,48.37984,5.509729,1.0,54.28121,4.633333,0.0,66.99,5.3,0.0,0.060049,3.99122,0.0,75.73044,1.0,4.52
2005-01-01,45.85582,0.0,6.9,48.37984,5.509729,1.0,54.28121,4.633333,0.0,66.99,5.3,0.0,0.060049,3.99122,0.0,75.73044,1.0,4.52


In [16]:
aligned_inf_df = get_aligned_df(df, ["INF", "RECESS_", "EPU_"])
display(aligned_inf_df.head())

,EPU_CAN,INF_YoY_CAN,RECESS_CAN,INF_CAN,EPU_GER,INF_YoY_GER,RECESS_GER,INF_GER,EPU_JAP,INF_YoY_JAP,...,INF_YoY_USA,RECESS_USA,INF_USA,EPU_MEX,INF_YoY_MEX,RECESS_MEX,INF_MEX,EPU_CHI,INF_CHI,RECESS_CHI
date,,,,,,,,,,,,,,,,,,,,,
1995-01-01,194.27143,2.14876,1.0,0.6,107.05698,1.70616,1.0,2.3,95.93575,-0.127899,...,2.80542,1.0,2.804,0.328965,34.999271,1.0,10.226,192.91191,16.791225,1.0
1995-02-01,126.45159,NaN,1.0,1.9,133.15922,NaN,1.0,2.0,69.55904,NaN,...,NaN,1.0,2.863,NaN,NaN,1.0,14.310,193.98785,NaN,1.0
1995-03-01,162.71977,NaN,1.0,2.1,98.76192,NaN,1.0,1.9,102.55873,NaN,...,NaN,1.0,2.853,NaN,NaN,1.0,20.430,88.22704,NaN,1.0
1995-04-01,128.56147,NaN,1.0,2.5,78.15823,NaN,1.0,1.9,66.37704,NaN,...,NaN,1.0,3.053,0.000000,NaN,1.0,29.392,131.03471,NaN,1.0
1995-05-01,96.17798,NaN,1.0,2.9,57.67824,NaN,1.0,1.6,93.31389,NaN,...,NaN,1.0,3.186,NaN,NaN,1.0,34.152,177.09686,NaN,1.0


In [17]:
aligned_ip_df = get_aligned_df(df, ["IP_", "RECESS_", "EPU_"])
display(aligned_ip_df.head())

,EPU_CAN,IP_CAN,RECESS_CAN,EPU_GER,IP_GER,RECESS_GER,EPU_JAP,IP_JAP,RECESS_JAP,EPU_USA,IP_USA,RECESS_USA,EPU_MEX,IP_MEX,RECESS_MEX,EPU_CHI,IP_CHI,RECESS_CHI
date,,,,,,,,,,,,,,,,,,
1999-01-01,101.68860,88.61232,0.0,111.26028,76.374468,0.0,115.46795,101.6937,1.0,89.71,87.2205,0.0,0.119560,81.299974,0.0,123.66965,117.9,1.0
1999-02-01,67.56765,88.52818,0.0,104.85598,NaN,0.0,95.82669,NaN,1.0,65.29,87.7260,0.0,NaN,81.331438,0.0,98.32405,102.1,1.0
1999-03-01,30.09706,88.19736,0.0,65.63091,NaN,0.0,62.84663,NaN,1.0,56.36,87.9125,0.0,NaN,81.485697,0.0,82.27813,109.0,1.0
1999-04-01,45.12118,88.73580,0.0,95.32922,NaN,0.0,102.23624,NaN,1.0,50.69,88.1082,0.0,0.113792,82.192431,0.0,16.56603,109.1,1.0
1999-05-01,57.52396,88.57040,0.0,87.84144,NaN,0.0,66.22225,NaN,0.0,48.12,88.6483,0.0,NaN,81.495393,0.0,116.03939,108.9,1.0


In [18]:
aligned_ex_df = get_aligned_df(df, ["EX_", "RECESS_", "EPU_"])
display(aligned_ex_df.head())

,EPU_CAN,EX_CAN,RECESS_CAN,EPU_GER,EX_GER,RECESS_GER,EPU_JAP,EX_JAP,RECESS_JAP,EPU_USA,EX_USA,RECESS_USA,EPU_MEX,EX_MEX,RECESS_MEX,EPU_CHI,EX_CHI,RECESS_CHI
date,,,,,,,,,,,,,,,,,,
1995-01-01,194.27143,94289.3,1.0,107.05698,1.123233e+11,1.0,95.93575,-13100.2,0.0,136.11,906.086,1.0,0.328965,631711.1250,1.0,192.91191,1.579864e+06,1.0
1995-02-01,126.45159,NaN,1.0,133.15922,NaN,1.0,69.55904,NaN,0.0,134.42,NaN,1.0,NaN,NaN,1.0,193.98785,NaN,1.0
1995-03-01,162.71977,NaN,1.0,98.76192,NaN,1.0,102.55873,NaN,0.0,97.52,NaN,1.0,NaN,NaN,1.0,88.22704,NaN,1.0
1995-04-01,128.56147,92252.0,1.0,78.15823,1.134238e+11,1.0,66.37704,NaN,0.0,69.45,918.791,1.0,0.000000,606978.6875,1.0,131.03471,NaN,1.0
1995-05-01,96.17798,NaN,1.0,57.67824,NaN,1.0,93.31389,NaN,0.0,64.41,NaN,1.0,NaN,NaN,1.0,177.09686,NaN,1.0


In [19]:
aligned_im_df = get_aligned_df(df, ["IM", "RECESS_", "EPU_"])
display(aligned_im_df.head())

,EPU_CAN,IM_CAN,RECESS_CAN,EPU_GER,IM_GER,RECESS_GER,EPU_JAP,RECESS_JAP,IM_JAP,EPU_USA,IM_USA,RECESS_USA,EPU_MEX,IM_MEX,RECESS_MEX,EPU_CHI,IM_CHI,RECESS_CHI
date,,,,,,,,,,,,,,,,,,
1995-01-01,194.27143,75290.0,1.0,107.05698,104862.090604,1.0,95.93575,0.0,49811.9,136.11,985.774,1.0,0.328965,511181.8125,1.0,192.91191,1.057782e+06,1.0
1995-02-01,126.45159,NaN,1.0,133.15922,NaN,1.0,69.55904,0.0,NaN,134.42,NaN,1.0,NaN,NaN,1.0,193.98785,NaN,1.0
1995-03-01,162.71977,NaN,1.0,98.76192,NaN,1.0,102.55873,0.0,NaN,97.52,NaN,1.0,NaN,NaN,1.0,88.22704,NaN,1.0
1995-04-01,128.56147,75893.0,1.0,78.15823,107717.813732,1.0,66.37704,0.0,51306.4,69.45,995.136,1.0,0.000000,525908.6250,1.0,131.03471,NaN,1.0
1995-05-01,96.17798,NaN,1.0,57.67824,NaN,1.0,93.31389,0.0,NaN,64.41,NaN,1.0,NaN,NaN,1.0,177.09686,NaN,1.0


In [20]:
aligned_cci_df = get_aligned_df(df, ["CCI", "RECESS_", "EPU_"])
display(aligned_cci_df.head())

,EPU_CAN,RECESS_CAN,CCI_CAN,EPU_GER,RECESS_GER,EPU_JAP,RECESS_JAP,EPU_USA,RECESS_USA,CCI_USA,EPU_MEX,RECESS_MEX,EPU_CHI,RECESS_CHI,CCI_CHI,CCI_GER,CCI_JAP,CCI_MEX
date,,,,,,,,,,,,,,,,,,
2001-04-01,108.59222,1.0,100.316503,92.08272,0.0,122.14508,1.0,105.17,1.0,88.4,0.142096,1.0,107.18360,1.0,101.0534,100.41740,99.05951,104.7466
2001-05-01,106.47535,1.0,100.357234,73.97758,1.0,57.98580,1.0,96.31,1.0,92.0,NaN,1.0,95.76117,1.0,101.0944,100.10490,98.74326,104.0665
2001-06-01,50.36757,1.0,100.171292,69.78983,1.0,74.85385,1.0,70.57,1.0,92.6,NaN,1.0,54.47955,1.0,101.1520,99.74490,98.35301,103.9679
2001-07-01,83.31136,1.0,99.798546,106.67741,1.0,123.72077,1.0,91.90,1.0,92.4,0.000000,1.0,118.17540,1.0,101.2000,99.49746,98.02427,104.4164
2001-08-01,73.99868,1.0,99.567142,88.27507,1.0,78.84318,1.0,80.84,1.0,91.5,NaN,1.0,131.47923,1.0,101.1580,99.34834,97.86748,104.5311


In [21]:
aligned_exr_df = get_aligned_df(df, ["EXR", "RECESS_", "EPU_"])
display(aligned_exr_df.head())

,EPU_CAN,RECESS_CAN,EXR_CAN,EPU_GER,RECESS_GER,EXR_GER,EPU_JAP,RECESS_JAP,EXR_JAP,EPU_USA,RECESS_USA,EPU_MEX,EXR_MEX,RECESS_MEX,EPU_CHI,RECESS_CHI,EXR_CHI
date,,,,,,,,,,,,,,,,,
1995-01-01,194.27143,1.0,1.413225,107.05698,1.0,1.433132,95.93575,0.0,99.766000,136.11,1.0,0.328965,6.41942,1.0,192.91191,1.0,8.4384
1995-02-01,126.45159,1.0,1.400489,133.15922,1.0,NaN,69.55904,0.0,98.236842,134.42,1.0,NaN,NaN,1.0,193.98785,1.0,8.4316
1995-03-01,162.71977,1.0,1.407696,98.76192,1.0,NaN,102.55873,0.0,90.519565,97.52,1.0,NaN,NaN,1.0,88.22704,1.0,8.4269
1995-04-01,128.56147,1.0,1.376200,78.15823,1.0,NaN,66.37704,0.0,83.689500,69.45,1.0,0.000000,NaN,1.0,131.03471,1.0,8.4074
1995-05-01,96.17798,1.0,1.360873,57.67824,1.0,NaN,93.31389,0.0,85.112727,64.41,1.0,NaN,NaN,1.0,177.09686,1.0,8.3077


In [22]:
aligned_ys_df = get_aligned_df(df, ["YS", "RECESS_", "EPU_"])
display(aligned_ys_df.head())

,EPU_CAN,RECESS_CAN,EPU_GER,RECESS_GER,EPU_JAP,RECESS_JAP,EPU_USA,YS_USA,RECESS_USA,EPU_MEX,RECESS_MEX,EPU_CHI,RECESS_CHI,YS_CAN,YS_MEX,YS_JAP,YS_CHI,YS_GER
date,,,,,,,,,,,,,,,,,,
2006-03-01,61.46333,0.0,94.2129,0.0,66.70839,0.0,51.51,-0.01,0.0,NaN,0.0,84.2251,0.0,0.219565,0.68,1.033,0.918,0.527289
2006-03-01,61.46333,0.0,94.2129,0.0,66.70839,0.0,51.51,-0.01,0.0,NaN,0.0,84.2251,0.0,0.219565,0.68,1.033,0.918,0.488744
2006-03-01,61.46333,0.0,94.2129,0.0,66.70839,0.0,51.51,-0.01,0.0,NaN,0.0,84.2251,0.0,0.219565,0.68,1.033,0.918,0.553674
2006-03-01,61.46333,0.0,94.2129,0.0,66.70839,0.0,51.51,-0.01,0.0,NaN,0.0,84.2251,0.0,0.219565,0.68,1.033,0.918,0.529344
2006-03-01,61.46333,0.0,94.2129,0.0,66.70839,0.0,51.51,-0.01,0.0,NaN,0.0,84.2251,0.0,0.219565,0.68,1.033,0.918,0.510848


In [23]:
aligned_ys_df = get_aligned_df(df, ["RECESS_", "EPU_"])
display(aligned_ys_df.head())

,EPU_CAN,RECESS_CAN,EPU_GER,RECESS_GER,EPU_JAP,RECESS_JAP,EPU_USA,RECESS_USA,EPU_MEX,RECESS_MEX,EPU_CHI,RECESS_CHI
date,,,,,,,,,,,,
1995-01-01,194.27143,1.0,107.05698,1.0,95.93575,0.0,136.11,1.0,0.328965,1.0,192.91191,1.0
1995-02-01,126.45159,1.0,133.15922,1.0,69.55904,0.0,134.42,1.0,NaN,1.0,193.98785,1.0
1995-03-01,162.71977,1.0,98.76192,1.0,102.55873,0.0,97.52,1.0,NaN,1.0,88.22704,1.0
1995-04-01,128.56147,1.0,78.15823,1.0,66.37704,0.0,69.45,1.0,0.000000,1.0,131.03471,1.0
1995-05-01,96.17798,1.0,57.67824,1.0,93.31389,0.0,64.41,1.0,NaN,1.0,177.09686,1.0


### Why Start at 1995?

Starting the analysis from 1995 ensures the widest coverage across all countries and variables, minimizing missing data and maximizing consistency in the dataset.


In [24]:
df_1995_onward = df[df.index >= "1995-01-01"] # Filter df to 1995 and onward

## Summaries

In [25]:
# Map prefixes to readable sheet names
prefix_labels = {
    "UNEMP": "Unemployment",
    "GDP_": "GDP",
    "GDPC": "GDPC",
    "INF": "Inflation",
    "IM": "Imports",
    "EX_": "Exports",
    "IP": "Industrial Prod",
    "YS": "Yield Spread",
    "CCI": "Consumer Conf",
    "EXR_": "Exchange Rate",
    "EPU": "EPU",
    "RECESS": "Recession Indicator"
}

### Descriptive Statistics by Variable

In the following cells, we generate summary statistics for each group of macroeconomic variables (e.g., GDP, inflation, unemployment). This helps us understand the distribution, scale, and variability of the data before performing deeper analysis or visualization.


In [26]:
# Generate and display all summaries, while saving them
summaries = {}
for prefix, label in prefix_labels.items():
    summary = describe_by_prefix(df, prefix)
    if summary is not None:
        summaries[label] = summary


Summary for prefix: UNEMP


,count,mean,std,min,25%,50%,75%,max
UNEMP_CAN,5892.0,6.94,1.38,2.80,6.10,6.90,7.40,14.20
UNEMP_GER,834.0,3.87,1.00,2.63,2.85,3.83,4.46,5.77
UNEMP_JAP,466.0,3.62,0.97,1.17,2.76,3.55,4.30,5.61
UNEMP_USA,5976.0,5.77,2.05,2.50,4.20,5.00,7.20,14.80
UNEMP_MEX,5487.0,4.02,0.88,2.10,3.38,3.82,4.82,7.60
UNEMP_CHI,443.0,4.55,0.30,2.37,4.52,4.56,4.63,5.00



Summary for prefix: GDP_


,count,mean,std,min,25%,50%,75%,max
GDP_CAN,1925.0,487918.68,97331.33,90980.80,457455.80,503123.50,552844.00,613548.00
GDP_GER,1805.0,7002.94,560.35,5145.15,6581.94,6973.64,7584.60,7705.84
GDP_JAP,440.0,530806.90,20147.37,446532.70,517963.00,529243.20,549481.25,557509.70
GDP_USA,1981.0,17537.56,4553.32,2172.43,16502.75,17860.45,20431.64,23542.35
GDP_MEX,1797.0,5504.35,557.95,3306.50,5059.04,5538.84,5980.91,6392.28
GDP_CHI,452.0,9582.54,4709.29,109.12,6241.30,9619.60,13493.44,17175.67



Summary for prefix: GDPC


...

,count,mean,std,min,25%,50%,75%,max
EXR_CAN,5710.0,1.20,0.14,0.96,1.06,1.24,1.32,1.60
EXR_GER,474.0,0.98,0.58,0.68,0.75,0.85,0.90,4.20
EXR_JAP,5710.0,115.68,32.89,76.64,101.78,110.03,119.51,358.02
EXR_MEX,183.0,8.77,4.88,0.01,7.01,10.90,11.13,13.51
EXR_CHI,5474.0,6.58,1.28,1.46,6.33,6.73,7.05,8.71



Summary for prefix: EPU


,count,mean,std,min,25%,50%,75%,max
EPU_CAN,5532.0,229.51,193.75,28.54,118.83,199.52,283.98,1635.40
EPU_GER,5436.0,252.77,237.89,28.43,111.59,168.61,278.87,1502.19
EPU_JAP,3177.0,102.68,35.71,29.92,74.16,98.58,126.78,204.73
EPU_USA,5542.0,124.78,80.32,38.21,74.71,103.79,140.79,560.88
EPU_MEX,1961.0,0.28,0.20,0.00,0.14,0.23,0.41,1.06
EPU_CHI,3862.0,194.19,161.02,9.07,84.23,136.72,260.26,935.31



Summary for prefix: RECESS


,count,mean,std,min,25%,50%,75%,max
RECESS_CAN,5156.0,0.33,0.47,0.0,0.0,0.0,1.0,1.0
RECESS_GER,5156.0,0.45,0.50,0.0,0.0,0.0,1.0,1.0
RECESS_JAP,5134.0,0.39,0.49,0.0,0.0,0.0,1.0,1.0
RECESS_USA,5312.0,0.35,0.48,0.0,0.0,0.0,1.0,1.0
RECESS_MEX,5134.0,0.25,0.43,0.0,0.0,0.0,0.0,1.0
RECESS_CHI,4941.0,0.66,0.47,0.0,0.0,1.0,1.0,1.0


In [28]:
with pd.ExcelWriter("../data/summary_tables/selected_summaries.xlsx") as writer:
    for sheet_name, summary_df in summaries.items():
        summary_df.to_excel(writer, sheet_name=sheet_name)
writer.book.close()

### Creating the Panel Dataset

We convert the cleaned, wide-format dataset into a long-format panel structure. Each row in the panel dataset represents a single observation for a specific country, variable, and month. This format is ideal for time series analysis, panel regressions, and comparative visualizations across countries and indicators.


- Melt wide-format DataFrame into long-format panel data

In [29]:
panel_df = df_1995_onward.reset_index().melt(
    id_vars="date", var_name="variable", value_name="value"
)

- Extract country code and base variable name

In [30]:
panel_df["country"] = panel_df["variable"].str.extract(r'_([A-Z]{3})$')
panel_df["variable_base"] = panel_df["variable"].str.extract(r'^([A-Z]+)')

- Drop rows with missing country or value

In [31]:
panel_df = panel_df.dropna(subset=["value", "country"])


- Normalize date to YYYY-MM and filter from 1995 onward

In [32]:
panel_df["date"] = pd.to_datetime(panel_df["date"])
panel_df = panel_df[panel_df["date"].dt.year >= 1995]
panel_df["date"] = panel_df["date"].dt.strftime("%Y-%m")

- Reorder and rename columns

In [33]:
panel_df = panel_df[["country", "date", "variable_base", "value"]]
panel_df.columns = ["Country", "Year", "Variable", "Value"]

panel_df.to_excel("../data/summary_tables/panel_dataset.xlsx", index=False)

### Country-Level Time Series Dataset

We generate a separate time series sheet for each country, with one column per macroeconomic variable. This structure makes it easier to visualize trends over time within a country and supports country-specific analysis such as forecasting or local policy evaluation.

In [36]:
# --- Extract all country codes from column suffixes (e.g., GDP_CAN → CAN) ---
countries = sorted({col.split("_")[-1] for col in df.columns if "_" in col})

# --- Save one sheet per country ---
output_path = "../data/summary_tables/country_aligned_timeseries.xlsx"
with pd.ExcelWriter(output_path) as writer:
    for country in countries:
        # Filter columns for the current country
        country_cols = [col for col in df.columns if col.endswith(f"_{country}")]
        if not country_cols:
            continue

        # Extract just the variable name (drop suffix)
        country_df = df_1995_onward[country_cols].copy()
        country_df.columns = [col.rsplit("_", 1)[0] for col in country_cols]

        # Ensure date index is formatted as YYYY-MM
        country_df.index = country_df.index.to_period("M").to_timestamp()
        country_df.index.name = "date"

        # Write to individual sheet
        country_df.to_excel(writer, sheet_name=country)
